In [2]:
import numpy as np
import scipy.stats as stats
import unittest


In [23]:


class BlockingSystemSimulation:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

    def average_service_time(self):
        return np.mean(self.service_times)

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            # Generate interarrival time and update arrival time
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]
            

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Generate service time
                service_time = np.random.exponential(self.mean_service_time)
                self.service_times.append(service_time)
                # Service can start immediately
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                # All service units are busy, customer is blocked
                self.blocked_customers += 1

            
           
    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        print(se)
        return p - z * se, p + z * se

import numpy as np
from scipy import stats
m = 10
mean_service_time = 8
mean_interarrival_time = 1
num_customers = 10000
n_cov = 100  # for estimating c
n_est = 100  # for final estimate

# --- Phase 1: Estimate c ---
X_train = []
Y_train = []

for _ in range(n_cov):
    sim = BlockingSystemSimulation(m, mean_service_time, mean_interarrival_time, num_customers)
    sim.simulate()
    X_train.append(sim.fraction_blocked())
    Y_train.append(sim.average_service_time())

X_train = np.array(X_train)
Y_train = np.array(Y_train)

cov_XY = np.cov(X_train, Y_train, ddof=1)[0, 1]
var_Y = 64/num_customers

print(var_Y)
c = -cov_XY / var_Y
print(f"Estimated c: {c:.4f}")

# --- Phase 2: Use c on new simulations ---
X_test = []
Y_test = []

for _ in range(n_est):
    sim = BlockingSystemSimulation(m, mean_service_time, mean_interarrival_time, num_customers)
    sim.simulate()
    X_test.append(sim.fraction_blocked())
    Y_test.append(sim.average_service_time())

X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_adj = X_test + c * (Y_test - mean_service_time)

# --- Statistics ---
def ci(sample, confidence=0.95):
    n = len(sample)
    mean = np.mean(sample)
    se = np.std(sample, ddof=1) / np.sqrt(n)
    z = stats.norm.ppf((1 + confidence) / 2)
    return mean - z * se, mean + z * se

# Mean and variance comparisons
print(f"Estimated c: {c:.4f}")
print(f"Naive mean blocked fraction: {np.mean(X_test):.4f}")
print(f"Adjusted mean blocked fraction: {np.mean(X_adj):.4f}")
print(f"Variance of naive: {np.var(X_test, ddof=1):.6f}")
print(f"Variance of adjusted: {np.var(X_adj, ddof=1):.6f}")
print(f"95% CI naive: {ci(X_test)}")
print(f"95% CI adjusted: {ci(X_adj)}")

0.0064
Estimated c: -0.0417
Estimated c: -0.0417
Naive mean blocked fraction: 0.1210
Adjusted mean blocked fraction: 0.1212
Variance of naive: 0.000032
Variance of adjusted: 0.000020
95% CI naive: (0.11992909550311792, 0.12213290449688208)
95% CI adjusted: (0.12033631495442841, 0.12208972031865876)


5.6

In [30]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd


class BlockingSystem:
    def __init__(self, m, mean_service_time, num_customers):
        self.m = m
        self.mean_service_time = mean_service_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_end_times = []

    def simulate(self, interarrival_times, service_times):
        arrival_time = 0
        self.blocked_customers = 0
        self.service_end_times = []

        for i in range(self.num_customers):
            arrival_time += interarrival_times[i]

            # Remove completed services
            self.service_end_times = [end for end in self.service_end_times if end > arrival_time]

            if len(self.service_end_times) < self.m:
                service_time = service_times[i]
                self.service_end_times.append(arrival_time + service_time)
            else:
                self.blocked_customers += 1

        return self.blocked_customers / self.num_customers


def generate_poisson_arrivals(U, mean_interarrival_time):
    return -np.log(U) * mean_interarrival_time


def generate_hyperexp_arrivals(U_choice, U_exp, p1=0.8, lmbda1=0.8333, p2=0.2, lmbda2=5.0):
    interarrivals = []
    for i in range(len(U_choice)):
        if U_choice[i] < p1:
            interarrivals.append(-np.log(U_exp[i]) / lmbda1)
        else:
            interarrivals.append(-np.log(U_exp[i]) / lmbda2)
    return np.array(interarrivals)


# Simulation parameters
m = 10
mean_service_time = 8
mean_interarrival_time = 1
num_customers = 10000
num_runs = 100

results = []

for _ in range(num_runs):
    # Generate common random numbers
    U_inter = np.random.uniform(size=num_customers)
    U_choice = np.random.uniform(size=num_customers)
    U_service = np.random.uniform(size=num_customers)

    poisson_arrivals = generate_poisson_arrivals(U_inter, mean_interarrival_time)
    hyperexp_arrivals = generate_hyperexp_arrivals(U_choice, U_inter)
    service_times = -np.log(U_service) * mean_service_time

    # With CRN
    system_poisson = BlockingSystem(m, mean_service_time, num_customers)
    system_hyper = BlockingSystem(m, mean_service_time, num_customers)
    fb_poisson = system_poisson.simulate(poisson_arrivals, service_times)
    fb_hyper = system_hyper.simulate(hyperexp_arrivals, service_times)
    diff_crn = fb_poisson - fb_hyper

    # Without CRN
    poisson_arrivals_indep = generate_poisson_arrivals(np.random.uniform(size=num_customers), mean_interarrival_time)
    hyperexp_arrivals_indep = generate_hyperexp_arrivals(np.random.uniform(size=num_customers),
                                                         np.random.uniform(size=num_customers))
    service_times_poisson = -np.log(np.random.uniform(size=num_customers)) * mean_service_time
    service_times_hyper = -np.log(np.random.uniform(size=num_customers)) * mean_service_time

    system_poisson_indep = BlockingSystem(m, mean_service_time, num_customers)
    system_hyper_indep = BlockingSystem(m, mean_service_time, num_customers)
    fb_poisson_indep = system_poisson_indep.simulate(poisson_arrivals_indep, service_times_poisson)
    fb_hyper_indep = system_hyper_indep.simulate(hyperexp_arrivals_indep, service_times_hyper)
    diff_indep = fb_poisson_indep - fb_hyper_indep

    results.append({
        "diff_crn": diff_crn,
        "diff_indep": diff_indep,
        "poisson blocking (using CRN)": fb_poisson,
        "hyperexp blocking (using CRN)": fb_hyper
    })

df_results = pd.DataFrame(results)

print(df_results.describe())

df_results.describe()


         diff_crn  diff_indep  poisson blocking (using CRN)  \
count  100.000000  100.000000                    100.000000   
mean    -0.016181   -0.017614                      0.122550   
std      0.003816    0.009353                      0.005987   
min     -0.027000   -0.037500                      0.106200   
25%     -0.018300   -0.023250                      0.118675   
50%     -0.016500   -0.018900                      0.122600   
75%     -0.014050   -0.011850                      0.126600   
max     -0.006800    0.010700                      0.136900   

       hyperexp blocking (using CRN)  
count                     100.000000  
mean                        0.138731  
std                         0.006506  
min                         0.123900  
25%                         0.134225  
50%                         0.139450  
75%                         0.143425  
max                         0.155700  


,diff_crn,diff_indep,poisson blocking (using CRN),hyperexp blocking (using CRN)
count,100.000000,100.000000,100.000000,100.000000
mean,-0.016181,-0.017614,0.122550,0.138731
std,0.003816,0.009353,0.005987,0.006506
min,-0.027000,-0.037500,0.106200,0.123900
25%,-0.018300,-0.023250,0.118675,0.134225
50%,-0.016500,-0.018900,0.122600,0.139450
75%,-0.014050,-0.011850,0.126600,0.143425
max,-0.006800,0.010700,0.136900,0.155700
